In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_community.document_loaders import TextLoader  # 텍스트 파일 로더
from langchain_openai.embeddings import OpenAIEmbeddings  # OpenAI 임베딩 사용
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain_community.vectorstores import FAISS  # 벡터 DB (FAISS) 사용

In [3]:
# 2. 벡터 데이터베이스 저장 경로 설정
DB_PATH = "./db/faiss_db"

# 3. 텍스트 파일을 로드하고 문서를 분할하는 함수 정의
def load_and_split_text(file_path, splitter):
    """
    주어진 텍스트 파일을 로드한 후, 설정된 Splitter를 사용하여 문서를 나누는 함수.
    
    Args:
        file_path (str): 로드할 파일 경로
        splitter (RecursiveCharacterTextSplitter): 텍스트 분할기 객체

    Returns:
        list: 분할된 문서 리스트
    """
    if not os.path.exists(file_path):
        print(f" 파일을 찾을 수 없습니다: {file_path}")
        return []
    
    try:
        loader = TextLoader(file_path, encoding="utf-8")  # 텍스트 파일 로드
        return loader.load_and_split(splitter)  # 분할하여 반환
    except Exception as e:
        print(f" 파일 로드 오류 ({file_path}): {e}")
        return []

# 4. 텍스트 분할기 설정 (600자 단위로 나누고, 100자 겹침 포함)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)

# 5. 두 개의 텍스트 파일 로드 및 분할
split_doc1 = load_and_split_text("data/ai-terminology.txt", text_splitter)
split_doc2 = load_and_split_text("data/finance-terminology.txt", text_splitter)

# 6. 문서 개수 출력
print(f"AI 문서 개수: {len(split_doc1)}")
print(f"금융 문서 개수: {len(split_doc2)}")

# 7. 모든 문서 합치기
all_documents = split_doc1 + split_doc2

# 8. FAISS 벡터 DB 생성 및 저장
try:
    # FAISS 벡터 DB 생성
    persist_db = FAISS.from_documents(
        documents=all_documents,
        embedding=OpenAIEmbeddings(),  # OpenAI Embeddings 사용
    )
    
    # 로컬 디스크에 저장
    if not os.path.exists(DB_PATH):
        os.makedirs(DB_PATH)
    persist_db.save_local(DB_PATH)
    
    print("FAISS 데이터베이스 저장 완료!")
except Exception as e:
    print(f" FAISS 데이터베이스 저장 오류: {e}")

# 9. 저장된 데이터 확인
try:
    # FAISS에서 인덱스 정보 확인
    print(f" 저장된 벡터 개수: {persist_db.index.ntotal}")
    print(f" 벡터 차원: {persist_db.index.d}")
    print(f" 인덱스 타입: {type(persist_db.index)}")
except Exception as e:
    print(f" 데이터 조회 오류: {e}")

AI 문서 개수: 6
금융 문서 개수: 5
FAISS 데이터베이스 저장 완료!
 저장된 벡터 개수: 11
 벡터 차원: 1536
 인덱스 타입: <class 'faiss.swigfaiss_avx2.IndexFlatL2'>


In [4]:
# 10. 유사도 검색 함수 정의
def search_query(query, k=2):
    """
    사용자 입력(query)에 대해 가장 유사한 문서를 검색하는 함수.

    Args:
        query (str): 검색할 문장 (예: "Transformer 개념 설명")
        k (int, optional): 검색할 문서 개수. Defaults to 2.

    Returns:
        None: 검색 결과를 출력
    """
    try:
        results = persist_db.similarity_search(query, k=k)  # 유사도 검색 수행
        print(f"\n [Query]: {query}\n")
        for i, doc in enumerate(results):
            print(f"[Result {i+1}]: {doc.page_content[:300]}...\n")  # 검색 결과 출력
    except Exception as e:
        print(f" 검색 오류: {e}")

# 11. 검색 테스트 실행
search_query("Transformer 에 대해 설명해줘", k=2)
search_query("Hedge Fund 에 대해 설명해줘?", k=2)


 [Query]: Transformer 에 대해 설명해줘

[Result 1]: Token (토큰)

정의: 텍스트 데이터를 더 작은 단위(단어, 문자, 문장 등)로 나누는 과정.
예시: "AI는 혁신적이다"를 ["AI", "는", "혁신적", "이다"]로 분할.
연관 키워드: 토큰화, NLP, 텍스트 전처리

Transformer (트랜스포머)

정의: 자연어 처리에서 사용되는 신경망 아키텍처로, 병렬 연산과 장기 의존성 처리가 강점.
예시: GPT, BERT 등의 모델이 트랜스포머 기반으로 동작함.
연관 키워드: 딥러닝, 자기 주의 메커니즘, NLP

Self-Attention (자기 주의 메커니즘)

정의...

[Result 2]: Earnings Per Share (주당순이익, EPS)

정의: 기업이 벌어들인 순이익을 총 발행 주식 수로 나눈 값.
예시: 애플의 EPS 증가로 인해 주가 상승 기대감이 커짐.
연관 키워드: 기업 실적, 투자 판단, 주식 분석

Interest Rate (이자율)

정의: 돈을 빌릴 때 지불해야 하는 비용의 비율.
예시: 연방준비제도(Fed)가 기준금리를 인상하면 대출 이자율도 상승함.
연관 키워드: 중앙은행, 채권 시장, 경제 정책

Inflation (인플레이션)

정의: 물가가 전반적으로 지속해서 상승하는 경제 현상.
예...


 [Query]: Hedge Fund 에 대해 설명해줘?

[Result 1]: Mutual Fund (뮤추얼 펀드)

정의: 여러 투자자의 자금을 모아 다양한 자산에 투자하는 펀드.
예시: 뮤추얼 펀드는 분산 투자로 위험을 줄이는 데 도움을 줌.
연관 키워드: 간접 투자, 포트폴리오, 펀드 매니저

Hedge Fund (헤지펀드)

정의: 공격적인 투자 전략을 사용하여 높은 수익을 추구하는 투자 펀드.
예시: 헤지펀드는 공매도, 레버리지 등 다양한 전략을 활용함.
연관 키워드: 고위험 투자, 적극적 운용, 레버리지

Asset Allocation (자산 배분)

정의: 투자 포트폴리오

In [6]:
# 12. 저장된 FAISS DB 로드 테스트
print("\n" + "="*50)
print("저장된 FAISS DB 로드 테스트")
print("="*50)

try:
    # 저장된 FAISS DB 로드
    loaded_db = FAISS.load_local(
        DB_PATH, 
        OpenAIEmbeddings(),
        allow_dangerous_deserialization=True
    )
    print("FAISS 데이터베이스 로드 완료!")
    
    # 로드된 DB로 검색 테스트
    print(f"로드된 벡터 개수: {loaded_db.index.ntotal}")
    
    # 검색 테스트
    test_results = loaded_db.similarity_search("인공지능이란?", k=1)
    print(f"\n로드된 DB 검색 결과: {test_results[0].page_content[:200]}...")
    
except Exception as e:
    print(f"FAISS DB 로드 오류: {e}")

# 13. 유사도 점수와 함께 검색
print("\n" + "="*50)
print("유사도 점수와 함께 검색")
print("="*50)

def search_with_score(query, k=2):
    """
    유사도 점수와 함께 검색하는 함수
    """
    try:
        results = persist_db.similarity_search_with_score(query, k=k)
        print(f"\n [Query]: {query}\n")
        for i, (doc, score) in enumerate(results):
            print(f"[Result {i+1}] (Score: {score:.4f}):")
            print(f"{doc.page_content[:200]}...\n")
    except Exception as e:
        print(f" 점수 검색 오류: {e}")

# 점수와 함께 검색 테스트
search_with_score("딥러닝과 머신러닝", k=3)


저장된 FAISS DB 로드 테스트
FAISS 데이터베이스 로드 완료!
로드된 벡터 개수: 11

로드된 DB 검색 결과: Explainable AI (설명 가능한 AI, XAI)

정의: AI의 의사결정 과정이 사람에게 이해할 수 있도록 설명되는 기술.
예시: AI가 의료 진단을 내릴 때, 왜 그런 결론을 내렸는지 설명함.
연관 키워드: 신뢰성, 모델 해석, AI 윤리

AGI (Artificial General Intelligence, 범용 인공지능)

정의: 특정 작업이 ...

유사도 점수와 함께 검색

 [Query]: 딥러닝과 머신러닝

[Result 1] (Score: 0.3324):
Zero-shot Learning (제로샷 학습)

정의: 특정 태스크에 대한 데이터 없이도 기존 학습된 지식을 활용하여 예측하는 AI 기술.
예시: AI가 새로운 언어를 학습하지 않았지만, 문맥을 기반으로 번역 가능.
연관 키워드: 전이 학습, 일반화 능력, NLP

Few-shot Learning (퓨샷 학습)

정의: 적은 양의 데이터로도 새로운 작업...

[Result 2] (Score: 0.3567):
Diffusion Model (확산 모델)

정의: 이미지 생성 AI에서 노이즈를 점진적으로 제거하며 데이터를 생성하는 기법.
예시: Stable Diffusion이 텍스트에서 이미지를 생성하는 과정.
연관 키워드: 이미지 생성, 딥러닝, 생성 AI

Hallucination (환각)

정의: AI가 실제 존재하지 않는 정보나 잘못된 내용을 생성하는 현상....

[Result 3] (Score: 0.3711):
Large Language Model (대형 언어 모델, LLM)

정의: 대량의 텍스트 데이터를 학습한 자연어 처리 모델.
예시: GPT-4, PaLM 등이 대형 언어 모델에 해당.
연관 키워드: 자연어 처리, 트랜스포머, AI 모델

Vector Database (벡터 데이터베이스)

정의: 데이터를 벡터 형태로 저장하고 유사도를 기반으로 검색

In [7]:
# 14. FAISS 인덱스 정보 출력
print("\n" + "="*50)
print("FAISS 인덱스 상세 정보")
print("="*50)

try:
    index = persist_db.index
    print(f"인덱스 타입: {type(index).__name__}")
    print(f"총 벡터 수: {index.ntotal}")
    print(f"벡터 차원: {index.d}")
    print(f"훈련 여부: {index.is_trained}")
    print(f"메트릭 타입: {index.metric_type}")
    
    # 메모리 사용량 (추정)
    memory_usage = index.ntotal * index.d * 4 / (1024 * 1024)  # 4 bytes per float, MB 단위
    print(f"추정 메모리 사용량: {memory_usage:.2f} MB")
    
except Exception as e:
    print(f"인덱스 정보 조회 오류: {e}")


FAISS 인덱스 상세 정보
인덱스 타입: IndexFlatL2
총 벡터 수: 11
벡터 차원: 1536
훈련 여부: True
메트릭 타입: 1
추정 메모리 사용량: 0.06 MB


### Vector Store Retriever

In [8]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document

# 문서 리스트 생성
documents = [
    Document(page_content="LangChain은 LLM 기반 애플리케이션을 쉽게 구축할 수 있도록 지원합니다."),
    Document(page_content="벡터 저장소는 문서를 벡터화하여 빠른 검색을 가능하게 합니다."),
    Document(page_content="멀티 쿼리 검색은 한 가지 질문을 여러 방식으로 변형하여 검색 효율을 높입니다."),
    Document(page_content="LangChain은 해리슨 체이스(Harrison Chase)가 2022년 10월에 개발되었습니다."),
    Document(page_content="LangChain의 주요 기능 중 하나는 다양한 LLM 모델과의 연동입니다."),
    Document(page_content="LangChain Core 라이브러리는 기본 추상화 및 LangChain 표현 언어"),
    Document(page_content="LangChain Community 라이브러리는 서드파티 통합"),
]

# 임베딩 모델 설정
embedding_model = OpenAIEmbeddings()

# FAISS 기반 벡터 저장소 생성
vector_store = FAISS.from_documents(documents, embedding_model)

# search_kwargs의 기본값은 {"k": 4} 기본적으로 최대 4개의 문서를 검색
# 필요에 따라 k 값을 조정하여 검색 문서 개수를 변경할 수 있음
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# 검색 실행
query = "LangChain 이란?"
retrieved_docs = retriever.invoke(query)

# 검색된 문서 출력
for doc in retrieved_docs:
    print(doc.page_content)

C:\Users\user\AppData\Local\Temp\ipykernel_15132\1118901403.py:17: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()


LangChain은 LLM 기반 애플리케이션을 쉽게 구축할 수 있도록 지원합니다.
LangChain의 주요 기능 중 하나는 다양한 LLM 모델과의 연동입니다.
LangChain은 해리슨 체이스(Harrison Chase)가 2022년 10월에 개발되었습니다.


### Multi Query Retriever

In [10]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

# 문서 리스트 생성
documents = [
    Document(page_content="LangChain은 LLM 기반 애플리케이션을 쉽게 구축할 수 있도록 지원합니다."),
    Document(page_content="벡터 저장소는 문서를 벡터화하여 빠른 검색을 가능하게 합니다."),
    Document(page_content="멀티 쿼리 검색은 한 가지 질문을 여러 방식으로 변형하여 검색 효율을 높입니다."),
    Document(page_content="LangChain은 해리슨 체이스(Harrison Chase)가 2022년 10월에 개발되었습니다."),
    Document(page_content="LangChain의 주요 기능 중 하나는 다양한 LLM 모델과의 연동입니다."),
    Document(page_content="LangChain Core 라이브러리는 기본 추상화 및 LangChain 표현 언어"),
    Document(page_content="LangChain Community 라이브러리는 서드파티 통합"),
]

# 임베딩 및 벡터스토어 설정
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embedding_model)

# LLM 모델 설정 (GPT 사용)
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

# Multi Query Retriever 생성
retriever = MultiQueryRetriever.from_llm(retriever=vector_store.as_retriever(), llm=llm)

# 검색 실행
query = "LangChain 이란?"
retrieved_docs = retriever.get_relevant_documents(query)

# 검색된 문서 출력
for doc in retrieved_docs:
    print(doc.page_content)

LangChain은 LLM 기반 애플리케이션을 쉽게 구축할 수 있도록 지원합니다.
LangChain의 주요 기능 중 하나는 다양한 LLM 모델과의 연동입니다.
LangChain Core 라이브러리는 기본 추상화 및 LangChain 표현 언어
LangChain은 해리슨 체이스(Harrison Chase)가 2022년 10월에 개발되었습니다.
